# Wine type prediction

In [1]:
import pandas as pd
import random
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
from sklearn import model_selection,linear_model

In [2]:
winedataset = pd.read_csv("winequalityN.csv").dropna()
winedataset

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.270,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,white,6.3,0.300,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,white,8.1,0.280,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,white,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,white,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6491,red,6.8,0.620,0.08,1.9,0.068,28.0,38.0,0.99651,3.42,0.82,9.5,6
6492,red,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
6494,red,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
6495,red,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [3]:
winedataset.groupby("type").size()

type
red      1593
white    4870
dtype: int64

Since these two sample size have a large difference therefore I will force these two difference types have the same sample size. which I will random select 1599 white wine sample from the 4898 samples

In [4]:
index = random.sample(list(range(0,4870,1)),1593)
white = winedataset[winedataset.type == "white"].iloc[index]
white
red = winedataset[winedataset.type == "red"]
red

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
4898,red,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
4899,red,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
4900,red,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
4901,red,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4902,red,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6491,red,6.8,0.620,0.08,1.9,0.068,28.0,38.0,0.99651,3.42,0.82,9.5,6
6492,red,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
6494,red,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
6495,red,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [5]:
finaldata = pd.concat([white,red])
finaldata

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
4566,white,8.6,0.360,0.26,11.1,0.030,43.5,171.0,0.99480,3.03,0.49,12.0,5
2077,white,7.0,0.220,0.39,2.1,0.055,39.0,198.0,0.99510,3.52,0.54,10.2,6
3077,white,6.2,0.250,0.38,7.9,0.045,54.0,208.0,0.99572,3.17,0.46,9.1,5
4517,white,6.8,0.170,0.17,5.1,0.049,26.0,82.0,0.99300,3.00,0.38,9.8,6
1911,white,7.1,0.180,0.26,1.3,0.041,20.0,71.0,0.99260,3.04,0.74,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6491,red,6.8,0.620,0.08,1.9,0.068,28.0,38.0,0.99651,3.42,0.82,9.5,6
6492,red,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
6494,red,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
6495,red,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [6]:
y = finaldata["type"]
trainx, testx,trainy,testy = model_selection.train_test_split(finaldata,y,test_size = 0.3)
trainx = trainx.drop(columns = "type")
testx = testx.drop(columns = "type")

In [7]:
trainx

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
5698,7.2,0.610,0.08,4.0,0.082,26.0,108.0,0.99641,3.25,0.51,9.4,5
1507,7.8,0.260,0.74,7.5,0.044,59.0,160.0,0.99600,3.22,0.64,10.0,6
5762,7.2,0.620,0.06,2.7,0.077,15.0,85.0,0.99746,3.51,0.54,9.5,5
5151,7.7,0.775,0.42,1.9,0.092,8.0,86.0,0.99590,3.23,0.59,9.5,5
3409,6.2,0.230,0.35,0.7,0.051,24.0,111.0,0.99160,3.37,0.43,11.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...
604,6.3,0.260,0.49,1.5,0.052,34.0,134.0,0.99240,2.99,0.61,9.8,6
5469,6.2,0.360,0.24,2.2,0.095,19.0,42.0,0.99460,3.57,0.57,11.7,6
5486,5.0,0.420,0.24,2.0,0.060,19.0,50.0,0.99170,3.72,0.74,14.0,8
4697,6.7,0.350,0.32,9.0,0.032,29.0,113.0,0.99188,3.13,0.65,12.9,7


In [8]:
trainy = pd.get_dummies(trainy)


In [9]:
trainy = trainy.drop(columns = "red")


In [10]:
trainy

,white
5698,0
1507,1
5762,0
5151,0
3409,1
...,...
604,1
5469,0
5486,0
4697,1


In [11]:
testx

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
1599,6.7,0.330,0.49,1.6,0.167,20.0,94.0,0.99140,3.11,0.50,11.4,6
4277,7.2,0.220,0.35,5.5,0.054,37.0,183.0,0.99474,3.08,0.50,10.3,5
306,7.2,0.430,0.24,6.7,0.058,40.0,163.0,0.99500,3.20,0.41,9.9,5
6017,5.6,0.660,0.00,2.5,0.066,7.0,15.0,0.99256,3.52,0.58,12.9,5
2001,6.7,0.310,0.08,1.3,0.038,58.0,147.0,0.99220,3.18,0.46,10.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...
6461,7.2,0.695,0.13,2.0,0.076,12.0,20.0,0.99546,3.29,0.54,10.1,5
5302,7.7,0.690,0.05,2.7,0.075,15.0,27.0,0.99740,3.26,0.61,9.1,5
5965,11.1,0.310,0.53,2.2,0.060,3.0,10.0,0.99572,3.02,0.83,10.9,7
2298,6.3,0.410,0.18,3.5,0.027,23.0,109.0,0.99018,3.34,0.54,12.8,8


In [12]:
testy = pd.get_dummies(testy)

In [13]:
testy = testy.drop(columns= "red")

# Improve dataset 

In [14]:
trainx

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
5698,7.2,0.610,0.08,4.0,0.082,26.0,108.0,0.99641,3.25,0.51,9.4,5
1507,7.8,0.260,0.74,7.5,0.044,59.0,160.0,0.99600,3.22,0.64,10.0,6
5762,7.2,0.620,0.06,2.7,0.077,15.0,85.0,0.99746,3.51,0.54,9.5,5
5151,7.7,0.775,0.42,1.9,0.092,8.0,86.0,0.99590,3.23,0.59,9.5,5
3409,6.2,0.230,0.35,0.7,0.051,24.0,111.0,0.99160,3.37,0.43,11.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...
604,6.3,0.260,0.49,1.5,0.052,34.0,134.0,0.99240,2.99,0.61,9.8,6
5469,6.2,0.360,0.24,2.2,0.095,19.0,42.0,0.99460,3.57,0.57,11.7,6
5486,5.0,0.420,0.24,2.0,0.060,19.0,50.0,0.99170,3.72,0.74,14.0,8
4697,6.7,0.350,0.32,9.0,0.032,29.0,113.0,0.99188,3.13,0.65,12.9,7


In [15]:
train_new = trainx.drop(columns= ["quality","fixed acidity","volatile acidity","free sulfur dioxide","total sulfur dioxide","sulphates","pH"])
train_new

,citric acid,residual sugar,chlorides,density,alcohol
5698,0.08,4.0,0.082,0.99641,9.4
1507,0.74,7.5,0.044,0.99600,10.0
5762,0.06,2.7,0.077,0.99746,9.5
5151,0.42,1.9,0.092,0.99590,9.5
3409,0.35,0.7,0.051,0.99160,11.0
...,...,...,...,...,...
604,0.49,1.5,0.052,0.99240,9.8
5469,0.24,2.2,0.095,0.99460,11.7
5486,0.24,2.0,0.060,0.99170,14.0
4697,0.32,9.0,0.032,0.99188,12.9


In [16]:
lgm2 = linear_model.LinearRegression()
lgm2.fit(train_new, trainy)

LinearRegression()

In [17]:
lgm2.score(train_new,trainy)

0.8062542629676923

In [18]:
lasso2 = linear_model.Lasso()
lasso2.fit(train_new, trainy)

Lasso()

In [19]:
lasso2.score(train_new,trainy)

0.0

In [34]:
logit = linear_model.LogisticRegression(solver="sag")
logit.fit(trainx, trainy)

/Users/harryzhang/miniforge3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/harryzhang/miniforge3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(solver='sag')

In [35]:
logit.score(trainx,trainy)

0.9210762331838565

In [36]:
logit.score(testx,testy)

0.9100418410041841

## Improved model

In [23]:
lgm = linear_model.LinearRegression()
lgm.fit(trainx, trainy)

LinearRegression()

In [24]:
lgm.score(trainx,trainy)

0.8756212465065769

In [25]:
lgm.score(testx,testy)

0.8780346952925053

In [26]:
lasso = linear_model.Lasso()
lasso.fit(trainx, trainy)

Lasso()

In [27]:
lasso.score(testx,testy)

0.5839821592106047

In [32]:
lasso.score(trainx,trainy)

0.5949683098572052

In [29]:
logit = linear_model.LogisticRegression(solver="lbfgs")
logit.fit(trainx, trainy)

/Users/harryzhang/miniforge3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/harryzhang/miniforge3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [30]:
logit.score(trainx,trainy)

0.9730941704035875

In [31]:
logit.score(testx,testy)

0.9759414225941423

# reference
1. https://blog.csdn.net/qq_31279347/article/details/82795405
1. https://blog.csdn.net/wanglingli95/article/details/78887771
3. https://datacarpentry.org/python-socialsci/11-joins/index.html
4. https://www.cnblogs.com/yanjy-onlyone/p/11288098.html
5. https://www.geeksforgeeks.org/how-to-convert-categorical-data-to-binary-data-in-python/
6. https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
7. https://scikit-learn.org/stable/glossary.html#term-random_state